<a href="https://colab.research.google.com/github/bl4ckf0xk/ModelX_First_Order/blob/main/ModelX_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, recall_score, precision_score
import joblib

In [ ]:
!pip install shap
import shap

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Dementia Prediction Dataset.csv'
MODEL_OUT = '/content/dementia_nonmedical_model.pkl'
RANDOM_STATE = 42
TARGET = 'DEMENTED'
SUBJECT_ID = 'NACCID'  # change if your identifier column has a different name

In [ ]:
def load_data(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Data file not found at {path}. Upload it to Colab or set DATA_PATH correctly.")
    df = pd.read_csv(path)
    return df

In [ ]:
NON_MEDICAL_WHITELIST = [
    # Subject demographics
    'BIRTHYR', 'BIRTHMO', 'SEX', 'HISPANIC', 'HISPOR', 'HISPORX',
    'RACE', 'RACEX', 'RACESEC', 'RACESECX', 'RACETER', 'RACETERX',
    'EDUC', 'MARISTAT', 'PRIMLANG', 'PRIMLANX', 'RESIDENC', 'HANDED', 'NACCLIVS', 'INDEPEND',
    # Co-participant
    'INBIRYR', 'INBIRMO', 'INSEX', 'INHISP', 'INHISPOR', 'INHISPOX', 'INRACE', 'INRACEX', 'INRASEC', 'INRASECX', 'INRATER', 'INRATERX', 'INEDUC', 'INRELTO', 'INRELTOX', 'NEWINF',
    # Lifestyle
    'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'ALCOCCAS', 'ALCFREQ',
    # Visit metadata
    'NACCVNUM', 'NACCNVST', 'NACCAVST', 'NACCDAYS', 'NACCFDYS', 'PACKET', 'FORMVER', 'TELCOV', 'TELMOD',
    # Family history (non-genetic fields)
    'NACCFAM', 'NACCMOM', 'NACCDAD'
]

In [ ]:
print('Loading data...')
df = load_data(DATA_PATH)
print('Rows:', len(df), 'Columns:', len(df.columns))

Loading data...


/tmp/ipython-input-175837067.py:4: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Rows: 195196 Columns: 1024


In [ ]:
# Ensure target and subject id exist
if TARGET not in df.columns:
    raise KeyError(f"Target column '{TARGET}' not found in dataset.")
if SUBJECT_ID not in df.columns:
    raise KeyError(f"Subject ID column '{SUBJECT_ID}' not found in dataset.")

# Reduce to columns we care about (keep subject id and target)
available_features = [c for c in NON_MEDICAL_WHITELIST if c in df.columns]
print(f'Using {len(available_features)} non-medical features (of {len(NON_MEDICAL_WHITELIST)} whitelist)')

# Warn about missing whitelist columns
missing = set(NON_MEDICAL_WHITELIST) - set(available_features)
if missing:
    print('Warning: The following whitelist columns were not found in your CSV and will be skipped:')
    print(sorted(list(missing)))

keep_cols = [SUBJECT_ID, TARGET] + available_features
df = df[keep_cols].copy()

Using 55 non-medical features (of 55 whitelist)


Handle NACC special codes -> convert common missing codes to NaN

In [ ]:
# Typical NACC codes: -4 = Not applicable, -1 or 9 = Unknown, 88/99 = other/missing depending on variable
NA_CODES = [-4, -1, 8, 9, 88, 95, 96, 97, 98, 99, 999]
for v in df.columns:
    if df[v].dtype.kind in 'biufc':
        df[v] = df[v].replace(NA_CODES, np.nan)
    else:
        # for object/string fields, keep as-is and handle missing later
        df[v] = df[v].replace([str(x) for x in NA_CODES], np.nan)

# Also treat target code '9' explicitly as NaN (unknown)
df[TARGET] = df[TARGET].replace(9, np.nan)

# Drop rows where target is missing
df = df[df[TARGET].notna()].copy()
print('After dropping unknown target, rows:', len(df))

After dropping unknown target, rows: 195196


Feature engineering

In [ ]:
# AGE if present; compute age-at-visit if possible using BIRTHYR and an approximate VISITYR if available.
# If NACCAGE exists, prefer it. Otherwise derive AGE from BIRTHYR with caveat.
if 'NACCAGE' in df.columns:
    df['AGE'] = df['NACCAGE']
elif 'BIRTHYR' in df.columns and 'NACCVNUM' in df.columns:
    # cheaper approximation: assume visit year unknown; fallback to age buckets via EDUC if needed
    df['AGE'] = np.nan
else:
    df['AGE'] = np.nan

# Pack-years and simple booleans
if 'PACKSPER' in df.columns and 'SMOKYRS' in df.columns:
    df['PACK_YEARS'] = df['PACKSPER'] * df['SMOKYRS']
else:
    df['PACK_YEARS'] = np.nan

if 'TOBAC30' in df.columns and 'TOBAC100' in df.columns:
    df['EVER_SMOKE'] = ((df['TOBAC30'] == 1) | (df['TOBAC100'] == 1)).astype('Int64')
else:
    df['EVER_SMOKE'] = pd.Series([pd.NA] * len(df))

# Heavy alcohol heuristic
if 'ALCFREQ' in df.columns:
    # interpret codes: (user should adjust according to their codebook)
    # treat high frequency codes (e.g., weekly/daily) as heavy
    df['HEAVY_ALCOHOL'] = df['ALCFREQ'].apply(lambda x: 1 if (pd.notna(x) and float(x) >= 5) else 0).astype('Int64')
else:
    df['HEAVY_ALCOHOL'] = pd.Series([pd.NA] * len(df))

# Years in study
if 'NACCDAYS' in df.columns:
    df['YEARS_IN_STUDY'] = df['NACCDAYS'] / 365.25
else:
    df['YEARS_IN_STUDY'] = np.nan

# Lives alone bool from NACCLIVS (if 1 = lives alone in your version)
if 'NACCLIVS' in df.columns:
    df['LIVES_ALONE'] = df['NACCLIVS'].apply(lambda x: 1 if x == 1 else 0).astype('Int64')
else:
    df['LIVES_ALONE'] = pd.Series([pd.NA] * len(df))

# Final feature list (keep engineered features)
engineered = ['AGE', 'PACK_YEARS', 'EVER_SMOKE', 'HEAVY_ALCOHOL', 'YEARS_IN_STUDY', 'LIVES_ALONE']
for e in engineered:
    if e not in df.columns:
        df[e] = np.nan

Final feature set building (numeric vs categorical)

In [ ]:
# Start from available_features (from whitelist) and add engineered numeric features
features = available_features.copy()
# Remove potential duplicates if present
for c in engineered:
    if c not in features:
        features.append(c)

# Remove subject id and target if present mistakenly
features = [f for f in features if f not in [SUBJECT_ID, TARGET]]

# Separate numeric and categorical heuristically
numeric_feats = []
categorical_feats = []
for f in features:
    if pd.api.types.is_numeric_dtype(df[f]) or f in engineered:
        numeric_feats.append(f)
    else:
        categorical_feats.append(f)

print('Numeric features:', len(numeric_feats))
print('Categorical features:', len(categorical_feats))

Numeric features: 50
Categorical features: 11


Preprocessing pipeline

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_feats),
        ('cat', categorical_transformer, categorical_feats)
    ],
    remainder='drop'
)

In [ ]:
X = df[features].copy()
y = df[TARGET].astype(int)
groups = df[SUBJECT_ID]

# Group-aware train-test split: ensure subjects do not leak across splits
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups=groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

print('Train subjects:', groups.iloc[train_idx].nunique(), 'Test subjects:', groups.iloc[test_idx].nunique())

Train subjects: 42029 Test subjects: 10508


Model pipeline (XGBoost) + hyperparameter search

In [ ]:
xgb_clf = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='auc',
    random_state=RANDOM_STATE,
    n_jobs=1
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('clf', xgb_clf)])

# Hyperparameter space for RandomizedSearchCV
param_dist = {
    'clf__n_estimators': [100, 250, 500],
    'clf__max_depth': [3, 5, 7],
    'clf__learning_rate': [0.01, 0.03, 0.1],
    'clf__subsample': [0.6, 0.8, 1.0],
    'clf__colsample_bytree': [0.6, 0.8, 1.0],
    'clf__gamma': [0, 1, 5]
}

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)

rsearch = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,
    scoring='roc_auc',
    cv=cv,
    verbose=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print('Starting hyperparameter search...')
rsearch.fit(X_train, y_train)
print('Best params:', rsearch.best_params_)

Starting hyperparameter search...
Fitting 4 folds for each of 20 candidates, totalling 80 fits


In [ ]:
best_model = rsearch.best_estimator_

# Predict probabilities and classes
y_test_prob = best_model.predict_proba(X_test)[:, 1]
y_test_pred = best_model.predict(X_test)

auc = roc_auc_score(y_test, y_test_prob)
rec = recall_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred)

print(f'Test AUC: {auc:.4f}')
print(f'Test Recall (sensitivity): {rec:.4f}')
print(f'Test Precision: {prec:.4f}')
print('\nClassification report:\n')
print(classification_report(y_test, y_test_pred))
print('\nConfusion matrix:\n')
print(confusion_matrix(y_test, y_test_pred))

Feature importance (approximate) - extract feature names after preprocessing

In [ ]:
preproc = best_model.named_steps['preprocessor']
num_names = numeric_feats
cat_names = []
if len(categorical_feats) > 0:
    ohe = preproc.named_transformers_['cat'].named_steps['onehot']
    # scikit-learn 1.0+ has get_feature_names_out
    try:
        cat_names = list(ohe.get_feature_names_out(categorical_feats))
    except Exception:
        # fallback for older versions
        cat_names = [f for f in categorical_feats]

feature_names = num_names + cat_names

try:
    importances = best_model.named_steps['clf'].feature_importances_
    fi = pd.Series(importances, index=feature_names).sort_values(ascending=False)
    print('\nTop 20 feature importances:')
    print(fi.head(20))
except Exception as e:
    print('Could not extract feature importances:', e)

In [ ]:
joblib.dump(best_model, MODEL_OUT)
print('Saved trained pipeline to', MODEL_OUT)

In [ ]:
def predict_subject(model_path, row_dict):
    """row_dict: dict mapping feature names to values (the same features used in training)
    Returns: dict with probability and class label (threshold 0.5)
    """
    model = joblib.load(model_path)
    X_in = pd.DataFrame([row_dict])
    prob = model.predict_proba(X_in)[:, 1][0]
    pred = int(prob > 0.5)
    return {'probability': float(prob), 'prediction': pred}

# Example usage (uncomment and replace example values):
# example = {f: None for f in features}
# example['SEX'] = 1
# example['EDUC'] = 12
# print(predict_subject(MODEL_OUT, example))